# Overview

This example demonstrates how to create and submit a job. We use [this]() endpoint to create and [this]() endpoint to submit the job.

1. Import required packages. Adjust [settings](../settings.ipynb) as necessary.

In [2]:
import json
import argparse
import nbimporter
nbimporter.options['only_defs'] = False

from settings import *
from endpoints.jobs import JobEndpoints

2. Create job config in JSON format. Adjust material ID, workflow ID and job name accordingly.

In [4]:
CONFIG = {
    "_material": {
        "_id": "tkmEX2KdrSzgo54km"
    },
    "workflow": {
        "_id": "6bd94da34794abf42a697fe1"
    },
    "name": "TEST JOB"
}

3. Initialize `JobEndpoints` class and call `create` function to create the job.

In [6]:
endpoint = JobEndpoints(HOST, PORT, ACCOUNT_ID, AUTH_TOKEN, VERSION, SECURE)
job = endpoint.create(CONFIG)

4. Submit job by its ID.

In [7]:
endpoint.submit(job['_id'])

4. Print the job in pretty JSON below. Check `status` field to make sure job is submiited. 

In [9]:
print json.dumps(endpoint.get(job['_id']), indent=4)

{
    "status": "active", 
    "_material": {
        "_id": "tkmEX2KdrSzgo54km", 
        "cls": "Material"
    }, 
    "compute": {
        "cluster": {
            "jid": "692.master-vagrant-cluster-001.exabyte.io", 
            "fqdn": "master-vagrant-cluster-001.exabyte.io"
        }, 
        "ppn": 1, 
        "queue": "D", 
        "timeLimit": "01:00:00", 
        "notify": "n", 
        "nodes": 1, 
        "arguments": {}
    }, 
    "name": "TEST JOB", 
    "workflow": {
        "exabyteId": "X6ZiGTAMo5bPrRK8Q", 
        "hash": "b4114aa5f2c0246d15c72709734d884c", 
        "name": "Total Energy", 
        "tags": [], 
        "creator": {
            "_id": "hY77F3hvNXy5RhepE", 
            "slug": "exadmin", 
            "cls": "User"
        }, 
        "owner": {
            "_id": "knJyjbqxww7kt4GpA", 
            "slug": "exadmin", 
            "cls": "Account"
        }, 
        "inSet": [
            {
                "_id": "c8bEk9FLFoLrbF7Dj", 
                "sl